In [2]:
import py21cmcast as p21c
import numpy as np 

from astropy import units

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

%load_ext autoreload
%autoreload 2

/home/users/g/f/gfacchin/exo21cmFAST_release/exo21cmFAST/src/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/users/g/f/gfacchin/exo21cmFAST_release/exo21cmFAST/src/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


## II. Define the grid of modes and redshifts

In [51]:
z_bins, k_bins = p21c.define_grid_modes_redshifts(6., 8 * units.MHz, z_max = 22, k_min = 0.1 / units.Mpc, k_max = 1 / units.Mpc)

fiducial = [None] * 2

fiducial[0]= p21c.Fiducial("../runs/CONSTRAINT_BKR", z_bins, k_bins, False, load = True)
fiducial[1] = p21c.Fiducial("../runs/CONSTRAINT_BKR_MINIHALOS", z_bins, k_bins, False, load = True)

for fid in fiducial:
    fid.observation = 'HERA'

In [54]:
## Classical sets of parameters
parameter_names_clas = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC',  'M_TURN', 't_STAR', 'L_X', 'NU_X_THRESH']
parameter_names_mini_1 = ['F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI',  't_STAR', 'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH']
parameter_names_mini_2 = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 'F_STAR7_MINI', 'ALPHA_STAR_MINI', 'F_ESC7_MINI',  'A_LW', 'L_X', 'NU_X_THRESH']

parameter_names = [parameter_names_clas, parameter_names_mini_1]
params = [[None] * len(parameter_names[i])  for i, _ in enumerate(fiducial)]

for ifid, fid in enumerate(fiducial):
    for iname, name in enumerate(parameter_names[ifid]) :  
        params[ifid][iname] = p21c.Parameter(fiducial=fid, name=name, verbose = False, plot = False, load=True)

Treating parameter F_STAR10
Treating parameter ALPHA_STAR
Treating parameter F_ESC10
Treating parameter ALPHA_ESC
Treating parameter M_TURN
Treating parameter t_STAR
Treating parameter L_X
Treating parameter NU_X_THRESH
Treating parameter F_STAR10
Treating parameter F_STAR7_MINI
Treating parameter ALPHA_STAR
Treating parameter ALPHA_STAR_MINI
Treating parameter t_STAR
Treating parameter F_ESC10
Treating parameter F_ESC7_MINI
Treating parameter ALPHA_ESC
Treating parameter L_X
Treating parameter L_X_MINI
Treating parameter NU_X_THRESH


In [60]:
extra_names  = ['m8.0', 'm8.0']
values        = [1e-29, 5e-29]
extra_params = [None] * len(extra_names)

for iname, extra_name in enumerate(extra_names) :  
    extra_params[iname] = p21c.Parameter(fiducial=fiducial[iname], name='DM_DECAY_RATE', verbose = False, plot=False, load=True, add_name = extra_name, values=values[iname])
    #extra_params[iname].plot_power_spectra(color=['b', 'k', 'r'])

Treating parameter DM_DECAY_RATE (m8.0)
Treating parameter DM_DECAY_RATE (m8.0)


## VI. Evaluate the Fisher matrix and its inverse

From the list of parameters objects defined above one can directly compute the Fisher matrix with the **evaluate\_fisher\_matrix()** function. One can add a modeling noise to the power spectrum with the attribute **frac\_noise** of the fiducial (as a percentage of the fiducial value). The triangle plot figure can be produced with the function **make\_triangle\_plot()** and plotted. Finally, the matrices can be displayed nicely with the **display\_matrix()** function.

In [71]:
sigma_tau = np.zeros(len(extra_names))

fisher_matrix = [None] * len(fiducial)
covariance_matrix = [None] * len(fiducial)

for iname, extra_name in enumerate(extra_names) : 
    fiducial[iname].frac_noise = 0.2
    fisher_matrix[iname]     = p21c.evaluate_fisher_matrix([*params[iname], extra_params[iname]])
    covariance_matrix[iname] = np.linalg.inv(fisher_matrix[iname]['matrix'])
    sigma_tau[iname]         = 1/(2.*np.sqrt(covariance_matrix[iname][-1][-1]))

covariance_matrix = np.array(covariance_matrix)

print('----------')
np.array(sigma_tau)

----------


array([3.06011727e+28, 1.48847148e+27])

In [1]:
%matplotlib inline
p21c.make_triangle_plot(covariance_matrix, [fish['name'] for fish in fisher_matrix], [fid.astro_params for fid in fiducial], color=['r'])

NameError: name 'p21c' is not defined

In [23]:
#parameter_names = ['F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI', 
#                   'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'NU_X_THRESH']

parameter_names = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 'F_STAR7_MINI', 'ALPHA_STAR_MINI',
                  'F_ESC7_MINI',  'A_LW', 'L_X', 'NU_X_THRESH']

params = [None] * len(parameter_names)

for iname, name in enumerate(parameter_names) :  
    params[iname] = p21c.Parameter(fiducial=fiducial, name=name, verbose = False, plot = False, load=True)

fiducial.frac_noise = 0.2

fisher_matrix     = p21c.evaluate_fisher_matrix(params)
covariance_matrix = np.linalg.inv(fisher_matrix['matrix'])
fig = p21c.make_triangle_plot(covariance_matrix, fisher_matrix['name'], fiducial.astro_params)
fig.savefig(fiducial.dir_path + '/triangle_plot.pdf')

Treating parameter F_STAR10
Treating parameter ALPHA_STAR
Treating parameter F_ESC10
Treating parameter ALPHA_ESC
Treating parameter F_STAR7_MINI
Treating parameter ALPHA_STAR_MINI
Treating parameter F_ESC7_MINI
Treating parameter A_LW
Treating parameter L_X
Treating parameter NU_X_THRESH
